# Azure AI Agent service - Single agent

<img src="agent.jpg" width=800>
<img src="foundry.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install --upgrade azure-ai-projects

In [2]:
import json
import os
import sys
import time

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [3]:
load_dotenv("azure.env")

True

In [4]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Project

In [5]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

In [6]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

## Single agent

In [7]:
model = "gpt-4o"
name = "gpt4o-agent"
instructions = "You are a GenAI agent"

agent = project_client.agents.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    temperature=0.7,
    top_p=1,
    response_format='auto',
)

print(f"Created agent with ID = {agent.id}")

Created agent with ID = asst_G3CWN3nJfcTrHrI0Gd0naby4


In [8]:
agent

{'id': 'asst_G3CWN3nJfcTrHrI0Gd0naby4', 'object': 'assistant', 'created_at': 1745961601, 'name': 'gpt4o-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a GenAI agent', 'tools': [], 'top_p': 1.0, 'temperature': 0.7, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

In [9]:
prompt = "Hello"

In [10]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_SCNWecNRuv0UkFpuCoj4QvQ9
Created message ID = msg_DgtW4JM8Zbd1ALCVzqANikII


In [11]:
thread

{'id': 'thread_SCNWecNRuv0UkFpuCoj4QvQ9', 'object': 'thread', 'created_at': 1745961601, 'metadata': {}, 'tool_resources': {}}

In [12]:
message

{'id': 'msg_DgtW4JM8Zbd1ALCVzqANikII', 'object': 'thread.message', 'created_at': 1745961602, 'assistant_id': None, 'thread_id': 'thread_SCNWecNRuv0UkFpuCoj4QvQ9', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [13]:
run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)

# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

print(run.status)

RunStatus.COMPLETED


In [14]:
run

{'id': 'run_gvsV3qVUIAnxAtQuTH8olmJD', 'object': 'thread.run', 'created_at': 1745961603, 'assistant_id': 'asst_G3CWN3nJfcTrHrI0Gd0naby4', 'thread_id': 'thread_SCNWecNRuv0UkFpuCoj4QvQ9', 'status': 'completed', 'started_at': 1745961604, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1745961604, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a GenAI agent', 'tools': [], 'tool_resources': {}, 'metadata': {}, 'temperature': 0.7, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 60, 'completion_tokens': 12, 'total_tokens': 72, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}

In [15]:
run.usage

{'prompt_tokens': 60, 'completion_tokens': 12, 'total_tokens': 72, 'prompt_token_details': {'cached_tokens': 0}}

In [16]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-04-29 21:20:04+0000
End     : 2025-04-29 21:20:04+0000
Elapsed : 0:00:00  (0.000 seconds)


In [17]:
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages)

{'object': 'list', 'data': [{'id': 'msg_glo960a7d3G58KdrtWP5QJVt', 'object': 'thread.message', 'created_at': 1745961604, 'assistant_id': 'asst_G3CWN3nJfcTrHrI0Gd0naby4', 'thread_id': 'thread_SCNWecNRuv0UkFpuCoj4QvQ9', 'run_id': 'run_gvsV3qVUIAnxAtQuTH8olmJD', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Hello! 😊 How can I assist you today?', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_DgtW4JM8Zbd1ALCVzqANikII', 'object': 'thread.message', 'created_at': 1745961602, 'assistant_id': None, 'thread_id': 'thread_SCNWecNRuv0UkFpuCoj4QvQ9', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_glo960a7d3G58KdrtWP5QJVt', 'last_id': 'msg_DgtW4JM8Zbd1ALCVzqANikII', 'has_more': False}


In [18]:
print(messages.data[0].content[0].text.value)

Hello! 😊 How can I assist you today?


## Another agent

In [19]:
prompt = "What is a GenAI model?"

In [20]:
# Initialize agent toolset with user functions
model = "gpt-4o"
name = "gpt4o-agent"
instructions = "You are a GenAI agent. Always answer in French"

agent = project_client.agents.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    temperature=0.7,
    top_p=1,
    response_format='auto',
)

print(f"Created agent with ID = {agent.id}")

Created agent with ID = asst_6WLp8Ze0TVWQdrAin2JlKNYv


In [21]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_4nMkiu2b2o14gBnICbmI4QHv
Created message ID = msg_Qr6SrtvYoaDYUWpgXmleZsqi


In [22]:
run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)

# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

print(run.status)

RunStatus.COMPLETED


In [23]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)

Un modèle GenAI (Intelligence Artificielle Générative) est un type de modèle d'intelligence artificielle conçu pour générer du contenu de manière autonome. Ces modèles utilisent des algorithmes avancés d'apprentissage automatique, souvent basés sur des architectures de réseaux neuronaux profonds, tels que les transformeurs (comme GPT, BERT, etc.).

Les modèles GenAI peuvent produire divers types de contenu : textes, images, vidéos, musique ou même code informatique, en fonction de leur entraînement. Leur fonctionnement repose sur l'analyse des données d'apprentissage pour comprendre les structures, les styles et les contextes, leur permettant de générer des résultats cohérents et pertinents.

Exemples d'applications des modèles GenAI :
- **Chatbots** : Répondre aux questions ou tenir une conversation fluide.
- **Création de contenu** : Rédiger des articles, des histoires ou des poèmes.
- **Conception visuelle** : Générer des images ou des designs.
- **Codage assisté** : Aider à écrire 

## Cleaning

In [24]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    print(agents.data[i])
    print()

{'id': 'asst_6WLp8Ze0TVWQdrAin2JlKNYv', 'object': 'assistant', 'created_at': 1745961606, 'name': 'gpt4o-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a GenAI agent. Always answer in French', 'tools': [], 'top_p': 1.0, 'temperature': 0.7, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

{'id': 'asst_G3CWN3nJfcTrHrI0Gd0naby4', 'object': 'assistant', 'created_at': 1745961601, 'name': 'gpt4o-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a GenAI agent', 'tools': [], 'top_p': 1.0, 'temperature': 0.7, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}



In [25]:
for i in range(len(agents.data)):
    print(f"Deleting agent {agents.data[i].id}")
    project_client.agents.delete_agent(agents.data[i].id)

Deleting agent asst_6WLp8Ze0TVWQdrAin2JlKNYv
Deleting agent asst_G3CWN3nJfcTrHrI0Gd0naby4


In [26]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    print(agents.data[i])
    print()